### **requirements**

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp -qqq

In [ ]:
!cd llama.cpp && GGML_CUDA=1 make && pip install -r requirements/requirements-convert_hf_to_gguf.txt -qqq

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE -DGGML_USE_CUDA -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include -DGGML_CUDA_USE_GRAPHS  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE 

In [ ]:
!pip install -Uqqqqqqqqq "huggingface_hub[cli]"

### **config**

In [ ]:
from google.colab import userdata
from huggingface_hub import login, snapshot_download
import os

HF_TOKEN = userdata.get('HF_TOKEN')
login(HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
name = 'gemma2-9b-it'
model_name = "google/gemma-2-9b-it"
methods = ['IQ1_M'] # here you'll find supported quant methods by llama.cpp https://github.com/ggerganov/llama.cpp/blob/master/examples/quantize/quantize.cpp
base_model = "./original_model/"
quantized_path = "./quantized_model/"
quant_name =f'{name}-{methods[0]}'
print(quant_name)

#### downloading base model & imatrix

In [ ]:
snapshot_download(repo_id=model_name, local_dir=base_model , local_dir_use_symlinks=False)
original_model = quantized_path+f'/FP16.gguf'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/22.2k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

(…)ransformers-4.42.0.dev0-py3-none-any.whl:   0%|          | 0.00/9.21M [00:00<?, ?B/s]

In [ ]:
!mkdir ./quantized_model/
!python llama.cpp/convert_hf_to_gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

INFO:hf-to-gguf:Loading model: original_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:Set model tokenizer
INFO:gguf.vocab:Setting special token type bos to 2
INFO:gguf.vocab:Setting special token type eos to 1
INFO:gguf.vocab:Setting special token type unk to 3
INFO:gguf.vocab:Setting special token type pad to 0
INFO:gguf.vocab:Setting add_bos_token to True
INFO:gguf.vocab:Setting add_eos_token to False
INFO:gguf.vocab:Setting chat_template to {{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if (message['role'] == 'assistant') %}{% set role = 'model' %}{% else %}{% set role = message['role'] %}{% endif %}{{ '<start_of_turn>' + role + '
' + messag

In [ ]:
# using imatrix already developed by https://huggingface.co/bartowski
!mkdir ./imatrix/
!huggingface-cli download bartowski/gemma-2-9b-it-GGUF --include "gemma-2-9b-it.imatrix" --local-dir ./imatrix/

Fetching 1 files:   0% 0/1 [00:00<?, ?it/s]Downloading 'gemma-2-9b-it.imatrix' to 'imatrix/.huggingface/download/gemma-2-9b-it.imatrix.cb2a0732cf3a4b668251a51e7bc6b1338119cde0dd213911bf0cf58bc005261b.incomplete'

gemma-2-9b-it.imatrix:   0% 0.00/6.12M [00:00<?, ?B/s]
gemma-2-9b-it.imatrix: 100% 6.12M/6.12M [00:00<00:00, 31.3MB/s]
Download complete. Moving file to imatrix/gemma-2-9b-it.imatrix
Fetching 1 files: 100% 1/1 [00:00<00:00,  1.70it/s]
/content/imatrix


### **quantization**

`llama-quantize --imatrix [imatrix-path] [fp-model-path] [quantized-output-path][method]`

In [ ]:
!./llama.cpp/llama-quantize --imatrix ./imatrix/gemma-2-9b-it.imatrix ./quantized_model/FP16.gguf ./quantized_model/gemma2-9b-it-IQ1_M.gguf IQ1_M

load_imatrix: imatrix dataset='/training_data/calibration_datav3.txt'
load_imatrix: loaded 294 importance matrix entries from ./imatrix/gemma-2-9b-it.imatrix computed on 128 chunks
prepare_imatrix: have 294 importance matrix entries
main: build = 3338 (3fd62a6b)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing './quantized_model/FP16.gguf' to './quantized_model/gemma2-9b-it-IQ1_M.gguf' as IQ1_M
llama_model_loader: loaded meta data with 29 key-value pairs and 464 tensors from ./quantized_model/FP16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma2
llama_model_loader: - kv   1:                               general.name str              = original_model
llama_model_loader: - kv   2:                      gemma2.context_length u32              = 8192
llama_model_loade

In [ ]:
# testing quantized model
!./llama.cpp/llama-cli -m .quantized_model/gemma2-9b-it-IQ1_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob.txt'

### **pushing .gguf to HF hub**

In [ ]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file

model_path = f"./quantized_model/{quant_name}.gguf"
print(model_path)
repo_name = f"{quant_name}-gguf"
repo_url = create_repo(repo_name, private=False)

./quantized_model/gemma2-9b-it-IQ1_M.gguf


In [ ]:
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo=f"{quant_name}.gguf",
    repo_id=f"valenradovich/{repo_name}",
    repo_type="model",
)

gemma2-9b-it-IQ1_M.gguf:   0%|          | 0.00/2.55G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/valenradovich/gemma2-9b-it-IQ1_M-gguf/commit/7517fd710eab8ff8dec4c2deb66f75fa2dac987e', commit_message='Upload gemma2-9b-it-IQ1_M.gguf with huggingface_hub', commit_description='', oid='7517fd710eab8ff8dec4c2deb66f75fa2dac987e', pr_url=None, pr_revision=None, pr_num=None)